# Function Calling
- Function Calling은 OpenAI API에서 특정 함수 정의(JSON schema 기반)를 주고, 모델이 해당 함수를 호출하는 형식의 응답을 생성하게 하는 기능입니다.
- 챗봇, 에이전트(Agent), 도구(Tool) 사용 등 다양한 자동화 및 플러그인 연동에 활용됩니다.

- 지원 모델 gpt-3-turbo, gpt-4, gpt-4-turbo  (2025.1 현재)

- 모델이 우리의 코드를 호출하도록 해서 우리의 함수들을 모델이 호출할수 있도록 하거나
- 모델이 우리가 원하는 특정 모양과 형식의 output 을 갖도록 강제할수 있다.

- https://platform.openai.com/docs/guides/function-calling?api-mode=chat





In [1]:
import os 

from dotenv import load_dotenv
load_dotenv()

print(f'OPENAI_API_KEY={os.getenv("OPENAI_API_KEY")[:20]}')

OPENAI_API_KEY=sk-proj-iKU13YeoxNgF


In [2]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

In [4]:
llm = ChatOpenAI(
    temperature=0.1, 
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")  # Who 맞다.

chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

print(response.content)


I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Rome.


In [ ]:
# 실시간으로 날씨를 가져올수 없다.
# 하지만!  이제 함수가 있다고 생각해보자.  실시간 데이터를 가져올 수 잇는 함수 말이다!
#  함수에 필요한 정보는 아마 '장소' 정보 뿐 일거다.  (위도, 경도..)

# ↓여기에 함수가 있다고 가정하자

In [ ]:
# ↓ 이러한 함수가 있다고 가정하자
#   위도,경도를 주면 실시간 날씨 정보를 리턴하는 함수..
def get_weather(lon, lat):
    print(f"call an api...lon:{lon}, lat:{lat}")

# ↓ LLM 에게 우리에게는 '이러이러한 동작' 을 해주는 함수가 있다고 알려주어야 한다
#   함수를 JSON schema 로 정의!

# 함수의 스키마

In [5]:
function_schema = {
    "name": "get_weather",  # 함수의 이름

    # description : 함수가 무슨 일을 하는지 기술
    # ↓'위도와 경도를 받아서 특정장소의 날씨정보를 가져오는 함수"
    "description": "function that takes longitude and latitude to find the weather of a place",  

    # 파라미터 기술
    "parameters": {
        "type": "object",
        "properties": {
            # 경도
            "lon": {"type": "string", "description": "The longitude coordinate",},
            # 위도
            "lat": {"type": "string","description": "The latitude coordinate",},
        },
    },

    # 필수 사항 기술
    "required": ["lon", 'lat'],  
}